In [2]:
import numpy as np

In [3]:
num_species = 53

J = np.zeros(num_species)
D = np.zeros(num_species)
n = np.zeros(num_species)
q = np.zeros(num_species)
mu = np.zeros(num_species)

E = 100

In [ ]:
J[i] = -D[i]*Nabla(n[i]) + q[i]/np.abs(q[i])*mu[i]*E*n[i]